In [1]:
#Import packages
import boto3
import json
import numpy as np
from numpy import argmax
import pyspark.sql.functions as F
import snowflake.connector
import pandas as pd
import spacy
import matplotlib.pyplot as plt
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from urllib.parse import urlparse
from langdetect import detect
from textblob import TextBlob
#RegEx
import regex as re

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
6,application_1616504514575_0007,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sm_client = boto3.client('secretsmanager',region_name='us-west-2')
secret_value = sm_client.get_secret_value(SecretId = 'emr_spark_qa')
username = json.loads(secret_value['SecretString'])['emr_spark_username']
password = json.loads(secret_value['SecretString'])['emr_spark_password']
#print(username)
#print(password)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#PYTHON connection info
conn = snowflake.connector.connect(
    account='powerreviews',
    database='analytics_prod',
    warehouse='LOWER_ENV_WAREHOUSE',
    user=username,
    password=password
    )


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
#Query some reviews
query_1 = "select distinct r.ugc_id, r.review_headline, r.review_comments, substr(r.locale,1,2) as lang, r.review_length, \
case when contains(r.observations, 'FL')  then 1 else 0 end as FL \
from analytics.review as r \
join analytics.merchant_group as mg \
on r.client_id = mg.client_id \
where contains(r.observations, 'PM') \
and not contains(r.observations, 'IA') and not contains(r.observations, 'IR') \
and mg.usage_type != 'Test' \
and r.created_date > '2020-01-01' \
limit 100000 \
"

#193267606, 324912854
#and r.created_date > '2020-01-01' \

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
reviews = pd.read_sql(query_1, conn)
reviews.head()
reviews.info()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   UGC_ID           100000 non-null  int64  
 1   REVIEW_HEADLINE  99999 non-null   object 
 2   REVIEW_COMMENTS  99484 non-null   object 
 3   LANG             100000 non-null  object 
 4   REVIEW_LENGTH    99484 non-null   float64
 5   FL               100000 non-null  int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 4.6+ MB

In [6]:
#Need to add "word" in front bc otherwise reviews with just emojis, symbols, #s, etc will not work in langdetect package
#Throws this error if no words: "langdetect.lang_detect_exception.LangDetectException: No features in text."
reviews['ALL_TEXT'] = reviews['REVIEW_HEADLINE'] + ' ' + reviews['REVIEW_COMMENTS']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [55]:
def onlywords(text):
    return re.sub(r'[^\w]', ' ', text) 


reviews['ALL_TEXT']=reviews['ALL_TEXT'].apply(str)
reviews['ALL_TEXT_2'] = reviews['ALL_TEXT'].apply(onlywords)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [56]:


from guess_language import guess_language
#guess_language("Ces eaux regorgent de renégats et de voleurs.")

#Need separate thing if less than ~20 characters
#https://pypi.org/project/guess_language-spirit/
#Need PyEnchant (spell checker with dictionaries) - not sure how to use this for detecting langs
#and how many dictionaries needed bc we support LOTS of different languages

#guess_language("good good good good good")
reviews['ALL_TEXT_2']=reviews['ALL_TEXT_2'].apply(str)
reviews['PRED_LANG'] = reviews['ALL_TEXT_2'].apply(guess_language)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [113]:
#Check if 1st and 2nd words are in English dictionary set
reviews['FIRST_WORD'] = reviews['ALL_TEXT_2'].str.split(' ').str[0]
reviews['FIRST_WORD'] = reviews['FIRST_WORD'].str.lower()
reviews['SECOND_WORD'] = reviews['ALL_TEXT_2'].str.split(' ').str[1]
reviews['SECOND_WORD'] = reviews['SECOND_WORD'].str.lower()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [114]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   UGC_ID           100000 non-null  int64  
 1   REVIEW_HEADLINE  99999 non-null   object 
 2   REVIEW_COMMENTS  99484 non-null   object 
 3   LANG             100000 non-null  object 
 4   REVIEW_LENGTH    99484 non-null   float64
 5   FL               100000 non-null  int64  
 6   ALL_TEXT         100000 non-null  object 
 7   PRED_LANG        100000 non-null  object 
 8   FIRST_WORD       100000 non-null  object 
 9   SECOND_WORD      99483 non-null   object 
 10  nltk_test        100000 non-null  bool   
 11  secondword_eng   100000 non-null  bool   
 12  firstword_eng    100000 non-null  bool   
 13  ALL_TEXT_2       100000 non-null  object 
 14  model_FL         100000 non-null  int64  
dtypes: bool(3), float64(1), int64(3), object(8)
memory usage: 9.4+ MB

In [115]:
from nltk.corpus import words
wordlist = words.words()
#reviews['FIRST_WORD']=reviews['FIRST_WORD'].apply(str)
reviews['nltk_test'] = reviews['FIRST_WORD'].isin(wordlist)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [116]:
import boto3
 
s3client = boto3.client('s3', region_name='us-west-2')
 
# These define the bucket and object to read
bucketname = 'pwr-analytics-emr' 
file_to_read = 'english_word_list.txt'

#Create a file object using the bucket and object key. 
fileobj = s3client.get_object(Bucket=bucketname, Key=file_to_read) 
# open the file object and read it into the variable filedata. 
filedata = fileobj['Body'].read()

# file data will be a binary stream.  We have to decode it 
eng_word_list = filedata.decode('utf-8') 

# Once decoded, you can treat the file as plain text if appropriate 
#print(contents)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [117]:
type(eng_word_list)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'str'>

In [118]:
len(eng_word_list)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4862992

In [119]:
reviews['firstword_eng'] = reviews['FIRST_WORD'].apply(lambda x : True if type(x) == str and x.isalpha() and x in eng_word_list else False)
reviews['secondword_eng'] = reviews['SECOND_WORD'].apply(lambda x : True if type(x) == str and x.isalpha() and x in eng_word_list else False)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [120]:
reviews.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   UGC_ID           100000 non-null  int64  
 1   REVIEW_HEADLINE  99999 non-null   object 
 2   REVIEW_COMMENTS  99484 non-null   object 
 3   LANG             100000 non-null  object 
 4   REVIEW_LENGTH    99484 non-null   float64
 5   FL               100000 non-null  int64  
 6   ALL_TEXT         100000 non-null  object 
 7   PRED_LANG        100000 non-null  object 
 8   FIRST_WORD       100000 non-null  object 
 9   SECOND_WORD      99483 non-null   object 
 10  nltk_test        100000 non-null  bool   
 11  secondword_eng   100000 non-null  bool   
 12  firstword_eng    100000 non-null  bool   
 13  ALL_TEXT_2       100000 non-null  object 
 14  model_FL         100000 non-null  int64  
dtypes: bool(3), float64(1), int64(3), object(8)
memory usage: 9.4+ MB

In [121]:
"website" in eng_word_list



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

False

In [159]:
"excelente" in eng_word_list



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

True

In [123]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

reviews.head(25)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

       UGC_ID                                               REVIEW_HEADLINE  \
0   310163689  BEAUTIFUL BAG                                                  
1   310053706  I love this bag -                                              
2   309950435  It was not broken                                              
3   310088124  I'll never use another company for my water dispenser needs    
4   309938472  Jerky Review                                                   
5   309635349  Beef Jerky Sticks Review                                       
6   310185959  Jerky Review                                                   
7   310336143  Best tea I have had....amazing                                 
8   309765207  Great support on my queries about compatible products.         
9   310150572  the good shizzles                                              
10  320860463  Good so far                                                    
11  310235362  Jerky Review                         

In [150]:
reviews['model_FL'] = 0

reviews.loc[reviews['LANG'] != reviews['PRED_LANG'] , 'model_FL'] = 1

#Our setup may have accidentally used "jp" as well as "ja" for Japenese
#"ja" is official code but mark "jp" as ok too
reviews.loc[ (reviews['LANG']=='jp') & (reviews['PRED_LANG']=='ja') , 'model_FL'] = 0

#'ca' is catalan which is close to spanish
reviews.loc[ (reviews['LANG']=='es') & (reviews['PRED_LANG']=='ca') , 'model_FL'] = 0


#reviews.loc[reviews['REVIEW_LENGTH'] < 40 , 'model_FL'] = 0

reviews.loc[ (reviews['LANG']=='en') & ((reviews['firstword_eng']==True) | (reviews['secondword_eng']==True)) , 'model_FL'] = 0

reviews.loc[reviews['PRED_LANG'] == 'UNKNOWN' , 'model_FL'] = 0

reviews.loc[reviews['PRED_LANG'] == 'af' , 'model_FL'] = 0

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [151]:
#reviews.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [152]:
FP = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 1)]
FN = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 0)]
TP = reviews[(reviews['FL'] == 1) & (reviews['model_FL'] == 1)]
TN = reviews[(reviews['FL'] == 0) & (reviews['model_FL'] == 0)]


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [153]:

FP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

754

In [154]:

FN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

151

In [155]:

TP['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

52

In [156]:
TN['UGC_ID'].count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

99043

In [157]:
FN.head(100)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

          UGC_ID  \
21     310237863   
45     309540163   
201    310026728   
355    310022368   
499    309977309   
933    310336875   
1692   319313881   
3375   319389636   
3413   319058075   
3510   319828692   
3883   319919732   
4111   319561892   
5052   319316379   
5492   319214305   
7741   329479955   
9891   321605376   
9988   321611593   
10269  321613709   
10478  321601179   
10584  321582797   
10699  321605345   
11599  321602784   
11612  321605753   
11876  265351132   
12144  265356742   
13818  319011154   
15088  319067801   
15143  319340819   
17096  319511993   
17457  319046225   
19020  319161601   
20021  329598920   
20481  309798454   
20522  309807013   
20527  310397766   
20567  310427580   
20893  310094830   
23970  318948277   
24046  320228227   
24082  319846146   
24351  320168736   
24782  320012365   
24959  319789525   
26179  324848835   
26185  324848987   
26994  319198660   
27523  319112442   
27785  329023693   
28101  329597998   


In [19]:
detect('good good stuff')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'cy'

In [28]:
FP.info()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6456 entries, 638 to 8681798
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   UGC_ID               6456 non-null   int64 
 1   NAME                 6456 non-null   object
 2   REVIEW_HEADLINE      6456 non-null   object
 3   REVIEW_COMMENTS      6456 non-null   object
 4   CR                   6456 non-null   int64 
 5   ALL_TEXT             6456 non-null   object
 6   NAME_1ST_PART_RAW    6456 non-null   object
 7   NAME_1ST_PART        6456 non-null   object
 8   ALL_TEXT_2           6456 non-null   object
 9   ALL_TEXT_3           6456 non-null   object
 10  ALL_TEXT_4           6456 non-null   object
 11  ALL_TEXT_5           6456 non-null   object
 12  ALL_TEXT_6           6456 non-null   object
 13  ALL_TEXT_7           6456 non-null   object
 14  regex_retailer       6456 non-null   object
 15  retailer_not_client  6456 non-null   bool  
 16  m

In [29]:
from pyspark.sql.types import *

mySchema = StructType([StructField("UGC_ID", StringType(), True)\
,StructField("NAME", StringType(), True)\
,StructField("REVIEW_HEADLINE", StringType(), True)\
,StructField("REVIEW_COMMENTS", StringType(), True)\
,StructField("CR", IntegerType(), True)\
,StructField("ALL_TEXT", StringType(), True)\
,StructField("NAME_1ST_PART_RAW", StringType(), True)\
,StructField("NAME_1ST_PART", StringType(), True)\
,StructField("ALL_TEXT_2", StringType(), True)\
,StructField("ALL_TEXT_3", StringType(), True)\
,StructField("ALL_TEXT_4", StringType(), True)\
,StructField("ALL_TEXT_5", StringType(), True)\
,StructField("ALL_TEXT_6", StringType(), True)\
,StructField("ALL_TEXT_7", StringType(), True)\
,StructField("regex_retailer", StringType(), True)\
,StructField("retailer_not_client", StringType(), True)\
,StructField("model_CR", IntegerType(), True)])



VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
SNOWFLAKE_SOURCE_NAME = "net.snowflake.spark.snowflake"

sfOptions = {
  'sfURL' : 'powerreviews.snowflakecomputing.com',
  'sfAccount' : 'powerreviews',
  'sfUser' : username,
  'sfPassword' : password,
  'sfDatabase' : 'ANALYTICS_WORKSHOP',
  'sfSchema' : 'DATA_SCIENCE',
  'sfWarehouse' : 'LOWER_ENV_WAREHOUSE'
}

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
#Turn into spark dataframe in order to write to Snowflake
#Couldn't get write_pandas() to work to write directly from pandas to Snowflake
FP = spark.createDataFrame(FP, schema=mySchema)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
FP.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions) \
    .option("dbtable","CR_FP_test4").mode("append").save()


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [109]:
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FP_TEST3")
conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TST_FN_TEST3")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SPARK6")
#conn.cursor().execute("drop table if exists ANALYTICS_WORKSHOP.DATA_SCIENCE.TEST_NLP_SWAP_TOKENS")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…